In [ ]:
import torch
import math
import numpy as np
import matplotlib.pyplot as plt

## 1.Pytorch Basics
## 2. Tensor Operations
## 3. Evaluating Derivatives
## 4. Linear Regression with Pytorch

Tensors are the PyTorch equivalent to Numpy arrays, with the addition to also have support for GPU acceleration (more on that later). The name “tensor” is a generalization of concepts you already know. For instance, a vector is a 1-D tensor, and a matrix a 2-D tensor. When working with neural networks, we will use tensors of various shapes and number of dimensions.

## 1.Pytorch Basics

In [ ]:
x = torch.tensor([2.,3.,6.])

In [ ]:
print(x)
print(x.shape)
print(x.type())

In [ ]:
A = torch.tensor([[3,4],
                 [9,5]])

In [ ]:
print(A)
print(A.shape)
print(A.type())

In [ ]:
print(A[0,1])
print(A[1,1])

We have tons of methods to create special types of tensors

In [ ]:
#axis=0
print(A[0])
print(A[1])

In [ ]:
x = torch.rand(3,3)
print(x)

In [ ]:
x = torch.randint(0,10, size=(4,4))
x

In [ ]:
x = torch.ones(4,4)
print(x)

In [1]:
# (Batch, Channel, Height, Width)

## 2. Tensor Operations

In [ ]:
A = torch.tensor([[3,4,7,7],
                  [9,5,9,1],
                  [-4,-3,5,0],
                  [-1,-2,5,1]])

In [ ]:
A[0,:]

In [ ]:
A[0:2,1:3]

In [ ]:
B = torch.tensor([[1,1,1,1],
                  [2,2,2,2],
                  [3,3,3,3]])
print(B)

Some unexpected results

In [ ]:
#sum over the elements of the first axis, which are the rows in fact.
B.sum(dim=0)

In [ ]:
print(B[0])
print(B[1])
print(B[2])
print('B[0]+B[1]+B[2] = ',B[0]+B[1]+B[2])

In [ ]:
B.sum(dim=1)

In [ ]:
print(B[0].sum())
print(B[1].sum())
print(B[2].sum())

Manipulating tensor with reshape, view, flatten

In [ ]:
B = torch.randint(10,(3,4))
print('Before reshape =:\n',B)
C = torch.reshape(B,(2,6))
print('\nAfter reshape:\n', C)


In [ ]:
#let the torch handle the other axis
C = B.view(-1,4)
print(C)

In [ ]:
B.flatten()

In [ ]:
B.view(-1)

One important operation includes "torch.argmax" and "torch.eq" operations. Imagine we are doing image classification with 4 classes 

$A = 0, B = 1 ,C = 2, D = 3$

What we do, we feed many images at the same time. Let's say we feed 2 class-B and 1 class-D. Our label array is

$$\text{label} = [1,1,3]$$

When we get the output from the model, it will look like;


$$
A = 
\begin{bmatrix}
12.2099 & 7.4075 & 1.9746,& 4.5719 \\
0.4014 & 19.8903 & 9.4084 & 9.3722 \\
3.3393 & 5.6968 & 8.2905 & 12.3987
\end{bmatrix}
$$

First row is the prediction for the first image and so on. Now, in each row, we get the index of largest entry and put this in an array using **"argmax"** command. This will be our prediction array. In this case, it must be

$$ \text{predictions} = [0,1,3]$$ 
Then we will count how many of the predictions is right. To do so, we will first Boolen compare labels and predictions using **"eq"** then count the True entries using **"sum"**. Carefully follow the example below

In [ ]:
# correct labels: A,B,A,C,D,A,C
label = torch.randint(0,4,size=(1,20))
label

In [ ]:
output = 10*torch.rand(20,4)
print(output)

In [ ]:
predictions = output.argmax(dim=1)
print(predictions)

In [ ]:
predictions.eq(label)

In [ ]:
number_of_correct = torch.sum(predictions.eq(label))
print(number_of_correct.item())

## 3. Evaluating Derivatives with Pytorch

**Ex-1**

Find the derivative of $f(x) = 3x^2$ at $x=1$. Note that the answer is 6.

In [ ]:
x = torch.tensor(1.,requires_grad = True)
y = 3*x**2
y.backward()
print(x.grad)

**Ex2**

Calculate the derivative of $f(x) = \big(x^2+ 7x + \cos(x)\big)^3 $ at $x=0$. Since $f'(x) = 3\big(x^2+ 7x + \cos(x)\big)^2\big(x+7-\sin(x)\big)$, we must have $f'(0)=21$ 

In [ ]:
x = torch.tensor(0.,requires_grad = True)
y = (x**2 + 7*x + torch.cos(x))**3
y.backward()
print(x.grad)

**Ex3**

Now, let's evaluate partial derivatives. $f(x,y) = x^2+y^2 + e^{x+y}\cos(xy)$ at $x=1,y=2$

In [ ]:
x = torch.tensor(1.,requires_grad = True)
y = torch.tensor(2.,requires_grad = True)
f = x**2 + y**2 + torch.exp(x+y)*torch.cos(x*y)
f.backward()
print('f_x: ',x.grad)
print("f_y: ",y.grad)

## 4. Linear Regression with Pytorch

Now, let's see how we do gradient decent with Pytorch. Our toy example is a simple linear regression problem. We will find the best line $$y=wx+c$$ to approximate a given set of data points $(x_i,y_i)_{i=1}^{N}$. Idea is to minimize the mean-square cost (MSE) function $$C=\dfrac{1}{N}\sum_{i=1}^{N}\Big(y(x_i)-y_i\Big)^2$$

Using chain rule, we can see that $$\dfrac{\partial C}{\partial w} = \dfrac{1}{N}\sum_{i=1}^{N}2\Big(y(x_i)-y_i\Big)x_i$$, $$ \dfrac{\partial C}{\partial b} = \dfrac{1}{N}\sum_{i=1}^{N}2\Big(y(x_i)-y_i\Big)1$$
Now we can go ahead and write the gradient decent for this problem. 

\begin{align}
    \begin{bmatrix}
           w_{n+1} \\
           b_{n+1} \\
     \end{bmatrix} =
     \begin{bmatrix}
           w_{n} \\
           b_{n} \\
      \end{bmatrix} -
      r\begin{bmatrix}
           \dfrac{\partial C}{\partial w} \\
          \dfrac{\partial C}{\partial b} \\
      \end{bmatrix}
\end{align}

We will have two approches;

- Implement naive graident decent with Pytorch
- Use optimizer provided by Pytorch

We can notice that Pytorch's implementation is way better in terms of error. That's why we must use the optimization tools provided in Pytorch

### a. Implement Naive Graident Decent with Pytorch

In [ ]:
def MSE(y,y_pred):
    return ((y-y_pred)**2).mean()

def model(w,b,x):
    return w*x+b

@torch.no_grad()
def DisplayResults(w,b,X_test,Y_test):
    y_preds = w * X_test + b
    plt.figure(figsize=(6,5))
    plt.plot(X_test,y_preds,'-b',X_test,Y_test,'ro',markersize = 2)
    plt.legend(['predicted','real_values'])
    plt.xlabel('x_values')
    plt.ylabel('y_values')
    plt.title('Linear_Regression_Model')
    plt.show()
    

In [ ]:
#Pick w = 7 and b = 3 and some noise
N = 300
X = torch.rand(N)
Y = 7*X + 3*torch.ones(N) + torch.rand(N)

#train-test split
X_train = X[0:math.ceil(N*0.8)]
X_test = X[math.ceil(N*0.8):]

Y_train = Y[0:math.ceil(N*0.8)]
Y_test = Y[math.ceil(N*0.8):]

In [ ]:
# initilize weight and bias
num_epochs = 400
lr = 1e-1
w = torch.randn(1,requires_grad=True)
b = torch.randn(1,requires_grad=True)
for i in range(num_epochs):
    #(1)FORWARD PASS
    
    #compute the predictions
    y_pred = model(w,b,X_train)
    
    #compute the loss
    loss = MSE(Y_train,y_pred)
    
    #(2)BACK-PROPOGATION    
    
    #compute the gradients
    loss.backward()
    with torch.no_grad():
        w.data = w.data - lr * w.grad
        b.data = b.data - lr * b.grad
        w.grad.zero_()
        b.grad.zero_()
    if i%100==0:
        print(f'w: {w.item():0.5f} b : {b.item():0.5f} loss : {loss.item():0.8f}')

In [ ]:
DisplayResults(w,b,X_test,Y_test)

### b. Implement  Graident Decent with Pytorch Optimizer

In [ ]:
#Better way of doing gradient decent. In fact, this is what we must do!
optimizer = torch.optim.Adam([w, b], lr = 0.1)
for i in range(num_epochs):
    #(1)FORWARD PASS
    
    #compute the predictions
    y_pred = model(w,b,X_train)
    
    #compute the loss
    loss = MSE(Y_train,y_pred)
    
    #(2)BACK-PROPOGATION    
    
    #compute the gradients
    loss.backward()
    
    #update the parameters
    optimizer.step()
    
    #detach the gradients
    optimizer.zero_grad()
    
    if i%100==0:
        print(f'w: {w.item():0.5f} b : {b.item():0.5f} loss : {loss.item():0.8f}')

In [ ]:
DisplayResults(w,b,X_test,Y_test)